In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
n_lines = 506
n_cols = 13
data = torch.tensor(())
data = data.new_zeros((n_lines,n_cols))
labels = torch.tensor(())
labels = labels.new_zeros((n_lines))
f = open('housing.csv')
n_line = 0
for line in f:
    cols = line.split()
    for i in range(n_cols):
        data[n_line][i] = float(cols[i])
    labels[n_line] = float( int( float(cols[n_cols]) ) )
    #labels[n_line] = float( int( float(cols[n_cols]) / 5) )  # цена с точностью до 5к долларов
    n_line += 1
    
f.close()

In [19]:
# варим фичи :)
# нормализуем параметры data
n_train_lines = 400
data_norm = torch.tensor(())
data_norm = data_norm.new_zeros((n_lines,n_cols))
data_min_max = torch.tensor(())
data_min_max = data_min_max.new_zeros((n_cols, 3))

# fuck the pandas
# find min and max
for i in range(n_cols):
    data_min_max[i][0] = 1000000  # set min
    data_min_max[i][1] = -1000000   # set max 

for i in range(n_lines):
    for j in range(n_cols):
        if data[i][j] < data_min_max[j][0]:
            data_min_max[j][0] = data[i][j]
        if data[i][j] > data_min_max[j][1]:
            data_min_max[j][1] = data[i][j]    

# calculate disperse
for i in range(n_cols):
    data_min_max[i][2] = data_min_max[i][1] - data_min_max[i][0]
    
# nomralize data, значения 0..1
for i in range(n_lines):
    for j in range(n_cols):
        data_norm[i][j] = (data[i][j] - data_min_max[j][0]) / data_min_max[j][2] * 1

data_train = torch.tensor(())
data_train = data_train.new_zeros((n_train_lines,n_cols))
labels_train = torch.tensor(())
labels_train = labels_train.new_zeros((n_train_lines))

data_train = data_norm[:n_train_lines]
labels_train = labels[:n_train_lines]

In [209]:
data_test = torch.tensor(())
data_test = data_test.new_zeros((n_lines-n_train_lines,n_cols))
labels_test = torch.tensor(())
labels_test = labels_test.new_zeros((n_lines-n_train_lines))

data_test = data_norm[n_train_lines:]
labels_test = labels[n_train_lines:]

In [210]:
batch_size = 10
dataset = TensorDataset(data_train, labels_train)
data_iter = DataLoader(dataset, batch_size, shuffle=True)

In [215]:
#model = torch.nn.Sequential(torch.nn.Linear(n_cols, 1))
model = torch.nn.Sequential(
    torch.nn.Linear(n_cols, 5), 
    #torch.nn.Sigmoid(),
    torch.nn.ReLU(True),
    #torch.nn.BatchNorm1d(5),
    torch.nn.Linear(5, 1),
)
loss = torch.nn.MSELoss(reduction='mean')
#trainer = torch.optim.SGD(model.parameters(), lr=0.00001,momentum=0.9,nesterov=True)
trainer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [216]:
num_epochs = 2000
for epoch in range(0, num_epochs):
    for X, y in data_iter:
        #print (X, y)
        trainer.zero_grad()
        l = loss(model.forward(X).reshape(-1), y)
        #print (f'loss={l}')
        #for par in model.parameters():
        #    print (par)
        #    break
        l.backward()
        trainer.step()
    l = loss(model.forward(data_train).reshape(-1), labels_train)
    ltest = loss(model(data_test).reshape(-1), labels_test)
    print (f'{int(l)}-{int(ltest)}', end=" ")
    #print('epoch %d, loss: %i, loss test: %i' % (epoch, l.item(), ltest.item()))
    #print('epoch %d, loss: %i, loss test: ' % (epoch, l.item()))
    #print('%i' % (l.item()), end=' ')
    #print('w', model[0].weight.data)
    #print('b', model[0].bias.data)
    
print (f'\nTrain loss: {loss(model(data_train).reshape(-1), labels_train)}')

641-253 624-243 603-229 571-209 522-179 451-136 357-84 257-42 177-26 129-37 106-58 97-76 94-86 91-91 90-91 88-90 87-87 85-85 84-82 83-79 81-79 80-75 79-72 78-69 77-66 75-63 74-61 73-58 72-57 71-56 70-52 69-49 68-48 68-46 67-44 66-43 65-41 64-41 64-39 63-37 62-36 61-34 61-33 60-32 59-31 59-30 58-29 57-28 57-28 56-27 56-26 55-25 55-25 54-24 54-24 53-24 53-23 52-23 52-22 51-22 51-21 50-21 50-21 50-21 49-20 49-20 48-20 48-20 48-20 47-19 47-19 47-19 46-19 46-19 46-19 45-19 45-19 45-19 44-19 44-19 44-19 44-19 43-19 43-19 43-19 42-19 42-19 42-19 42-19 41-19 41-19 41-19 41-19 40-19 40-19 40-19 40-19 39-19 39-19 39-19 39-19 38-19 38-19 38-19 38-19 38-19 37-19 37-19 37-19 37-19 36-19 36-19 36-19 36-19 36-19 36-19 35-19 35-19 35-19 35-19 35-19 34-19 34-19 34-19 34-19 34-19 34-19 33-19 33-19 33-19 33-19 33-19 33-19 33-19 32-19 32-19 32-19 32-19 32-19 32-19 32-19 31-19 31-18 31-18 31-18 31-18 31-18 31-18 31-18 30-18 30-18 30-18 30-18 30-18 30-18 30-18 30-18 30-18 30-18 29-18 29-18 29-18 29-18 29-18

In [217]:
#test model


l = loss(model(data_test).reshape(-1), labels_test)
print (f'Test loss = {l}\n')

for i in range(n_lines-n_train_lines):
    dat = data_test[i]
    print (model(dat), labels_test[i])

# batch_size = 1
# dataset_test = TensorDataset(data_test, labels_test)
# data_iter_test = DataLoader(dataset, batch_size, shuffle=True)

# for X, y in data_iter:
#     print (X,y)
#     #pp = model.forward(X).reshape(-1)
#     l = loss(model.forward(X).reshape(-1), y)
# #     break

Test loss = 18.67812728881836

tensor([11.4072], grad_fn=<AddBackward0>) tensor(5.)
tensor([15.6897], grad_fn=<AddBackward0>) tensor(7.)
tensor([16.1100], grad_fn=<AddBackward0>) tensor(12.)
tensor([12.9494], grad_fn=<AddBackward0>) tensor(8.)
tensor([8.2331], grad_fn=<AddBackward0>) tensor(8.)
tensor([7.8547], grad_fn=<AddBackward0>) tensor(5.)
tensor([10.2125], grad_fn=<AddBackward0>) tensor(11.)
tensor([21.5754], grad_fn=<AddBackward0>) tensor(27.)
tensor([13.1464], grad_fn=<AddBackward0>) tensor(17.)
tensor([19.4769], grad_fn=<AddBackward0>) tensor(27.)
tensor([19.7605], grad_fn=<AddBackward0>) tensor(15.)
tensor([16.0738], grad_fn=<AddBackward0>) tensor(17.)
tensor([5.0521], grad_fn=<AddBackward0>) tensor(17.)
tensor([11.7404], grad_fn=<AddBackward0>) tensor(16.)
tensor([2.0156], grad_fn=<AddBackward0>) tensor(7.)
tensor([9.5224], grad_fn=<AddBackward0>) tensor(7.)
tensor([12.1952], grad_fn=<AddBackward0>) tensor(7.)
tensor([8.4436], grad_fn=<AddBackward0>) tensor(10.)
tensor([6.0